In [1]:
import torch
import glob
import os
import pandas as pd
from tqdm import tqdm

In [2]:
data_root = 'out'
path_list = ['test','psa_80k']

pt_lists = []
for p in path_list:
    pt_list = sorted(glob.glob(f'{data_root}/{p}/*.pt'))
    pt_lists.append(pt_list)

os.makedirs(f'{data_root}/essamble', exist_ok=True)

for p in tqdm(zip(*pt_lists),total=len(pt_list)):
    logits = list(map(lambda x: torch.load(x),p))
    logit_sum = sum(logits)
    result = torch.argmax(logit_sum,dim=1)
    file_name = p[0].split('/')[-1].replace('.pt','.txt')
    with open(f'{data_root}/essamble/{file_name}', 'w') as f:
        for r in result[0]:
            r_list = r.tolist()
            f.write(' '.join(map(str,r_list)) + '\n')

624it [00:52, 11.78it/s]


In [3]:
print('\nmaking submission ...')
dic = {"image_id":[],"PredictionString":[]}
for path in tqdm(sorted(glob.glob(f'{data_root}/essamble/*.txt'))):
    image_id = path.replace(f'{data_root}/essamble/','').replace('_03_','_03/').replace('vt_','vt/').replace('.txt','.jpg')
    with open(path, 'r') as f:
        value_list = f.readlines()
        new_value_list = []
        for i in range(0,len(value_list),2):
            value = value_list[i].split(' ')
            value = ' '.join(value[0::2])
            new_value_list.append(value)

    dic["image_id"].append(image_id)
    dic["PredictionString"].append(' '.join(new_value_list))

df = pd.DataFrame(dic)
df.to_csv(f'{data_root}/essamble/_submission.csv',index=False)

  3%|▎         | 17/624 [00:00<00:03, 162.46it/s]


making submission ...


100%|██████████| 624/624 [00:04<00:00, 146.46it/s]
